In [16]:
import pandas as pd
import numpy as np

In [17]:
debug=0

⚠️ Attention à fournir un fichier XLSX

In [18]:
notesNom=u"/Users/gilles/Downloads/24-Morpho-L1.xlsx"

In [19]:
xls = pd.ExcelFile(notesNom)
xls.sheet_names

[u'Notes', u'eCampus']

In [20]:
dfNotes=pd.read_excel(notesNom,encoding="utf8")
dfNotes.columns

Index([u'Numéro d'identification', u'Nom de famille', u'Prénom',
       u'Adresse de courriel',
       u'Test Contrôle Continu : Analyse morphologique (1) (Brut)',
       u'Test Contrôle Continu : Analyse morphologique (2) (Brut)',
       u'Test Contrôle Continu : Bizarreries morphologiques (1) (Brut)',
       u'Test Contrôle Continu : Bizarreries morphologiques (2) (Brut)',
       u'Test Contrôle continu : Opérations phonologiques (Brut)',
       u'Test Contrôle continu : Constructions sémantiques (Brut)',
       u'Devoir Dépôt pour le travail final sur les kanoniks (Brut)',
       u'Quizz01-03 (Brut)', u'Quizz04 (Brut)', u'Quizz05 (Brut)',
       u'Quizz06 (Brut)', u'Quizz07a (Brut)', u'Quizz07b (Brut)',
       u'Quizz08 (Brut)', u'Dernier téléchargement depuis ce cours'],
      dtype='object')

In [21]:
dfNotes = pd.read_excel(xls, sheet_name=u'Notes',encoding="utf8")
dfNotes.set_index([u"Numéro d'identification"],inplace=True)
len(dfNotes.columns)

18

In [22]:
dfNotes.columns=[u'Nom', u'Prénom', u'Adresse de courriel',
                 u'Morpho1', u'Morpho2', u'Bizarre1', u'Bizarre2', 
                 u'Phono', u'Sém', u'Devoir']+sorted([u"Q%02d"%(n+1) for n in range(2,9)])+["date"]

#### sélection des colonnes
- notesCC pour le contrôle continu
- notesQuizz pour les quizz en CM

In [23]:
notesCC=[u'Morpho1', u'Morpho2', u'Bizarre1', u'Bizarre2', u'Phono', u'Sém' ]
notesQuizz=sorted([u"Q%02d"%(n+1) for n in range(2,9)])
dfNotes[notesQuizz].head(10)

,Q03,Q04,Q05,Q06,Q07,Q08,Q09
Numéro d'identification,,,,,,,
436893,15,18,-,18,18,17,13
440340,13,13,12,13,15,13,12
435916,13,17,13,15,15,15,13
442931,-,-,-,-,-,-,-
438199,-,-,15,13,-,-,-
386382,-,-,-,-,-,-,-
440881,13,20,17,17,18,15,13
441950,-,-,-,13,-,-,-
437791,-,13,-,-,-,13,-


### Calcul des notes avec bonification
Toutes les notes du CC (*nbCC*) comptent et on inclue aussi les *nbCC* meilleures notes des Quizz en partant de la meilleure une par une
- *notes* contient toutes les moyennes calculées
- on retourne le max de *notes*

In [24]:
def moyenneCC(x):
    if debug: print x.name
    xNotesCC=sorted([n for n in x[notesCC].tolist() if not (n=="-" or np.isnan(n))],reverse=True)
    sommeNotes=float(sum(xNotesCC))
    nbNotes=len(notesCC)
    moyenneInitiale=round(sommeNotes/nbNotes+.25)
    return moyenneInitiale
        

In [25]:
def bonus(x):
    if debug: print x.name
    xNotesCC=sorted([n for n in x[notesCC].tolist() if not (n=="-" or np.isnan(n))],reverse=True)
    sommeNotes=float(sum(xNotesCC))
    nbNotes=len(notesCC)
    moyenneInitiale=round(sommeNotes/nbNotes+.25)
    xNotesBonus=sorted([n for n in x[notesQuizz].tolist() if not (n=="-" or np.isnan(n))],reverse=True)
    if debug: print "Quizz",xNotesBonus
    if debug: print "CC",xNotesCC
    if xNotesCC:
        notes=[]
        nbCC=len(xNotesCC)
        bonusNotes=xNotesBonus
        if len(bonusNotes)>nbCC:
            bonusNotes=bonusNotes[:nbCC]
        notes.append(moyenneInitiale)
        if debug: print "somme", sommeNotes,"moyenne brut",sommeNotes/nbNotes
        if debug: print "moyenne initiale",moyenneInitiale,"bonus",bonusNotes
        for b in bonusNotes:
            sommeNotes+=b
            nbNotes+=1
            nouvelleMoyenne=round(float(sommeNotes)/nbNotes+.25)
            notes.append(nouvelleMoyenne)
            if debug: print b,sommeNotes,nbNotes, nouvelleMoyenne
        if debug: print notes
        return max(notes)
    else:
        return np.nan
        

### Ajout de la colonne **Notes** à la structure et exportation
- apply(lambda x: bonus(x), axis=1) permet d'appliquer la fonction bonus à chaque ligne
 - la fonction bonus reçoit une ligne de données complète
 - les valeurs sont accessibles par colonne
- on ajoute 3 colonnes
 - Note pour la note de contrôle continu avec les bonus de quizz
 - NoteCC pour la note des exercices en ligne brute
 - Gain pour la différence entre les deux notes

In [26]:
dfNotes.loc[:,"Note"]=dfNotes.loc[:,notesCC+notesQuizz].apply(lambda x: bonus(x),axis=1)
dfNotes.loc[:,"NoteCC"]=dfNotes.loc[:,notesCC].apply(lambda x: moyenneCC(x),axis=1)

In [27]:
dfNotes["Gain"]=dfNotes["Note"]-dfNotes["NoteCC"]

In [28]:
dfNotes["Gain"].mean()

0.13414634146341464

In [29]:
dfNotes[dfNotes["Note"].notnull()][u"Prénom Nom Note NoteCC Gain".split(" ")]

,Prénom,Nom,Note,NoteCC,Gain
Numéro d'identification,,,,,
436893,GABRIEL,ABELA--FRANZETTI,17.0,17.0,0.0
440340,FLORELLE,ABERARD,17.0,17.0,0.0
435916,EULALIE,ADAM,15.0,15.0,0.0
438199,NESLIHAN,AKPINAR,17.0,17.0,0.0
440881,RACHEL,AMOUZOU,18.0,17.0,1.0
441950,FREDERICK JOHN,ANDERSON,13.0,12.0,1.0
437791,MAURINE,APPRIOU,18.0,18.0,0.0
440486,ELYA,AURICES,18.0,18.0,0.0
436384,CAMILLE,BALDY- - BOUILLEUX,19.0,19.0,0.0


In [30]:
with pd.ExcelWriter(notesNom,mode='a',engine='openpyxl') as writer: 
    dfNotes[u"Prénom Nom Note NoteCC Gain".split(" ")].to_excel(writer, sheet_name=u'eCampus',encoding="utf8")